## 12.4 Wstrzykiwanie interfejsów

W tym przykładzie przyjrzymy się procesowi wstrzyknięć interfejsów. W dalszym ciągu posługujemy się abstrakcją komputera - tym razem nasz komputer będzie składał się tylko z cpu. `CPU` jest interfejsem, mamy również dwie klasy implementujące ten interfejs - `AMD` i `Intel`.

In [ ]:
public interface CPU {
    String name();
}

public class Intel implements CPU {

    @Override
    public String name() {
        return "intel";
    }
}

public class AMD implements CPU {

    @Override
    public String name() {
        return "amd";
    }
}

public class Computer {
    private CPU cpu;

    @Inject
    public Computer(CPU cpu) {
        this.cpu = cpu;
    }

    public String cpuName(){
        return cpu.name();
    }
}

klasy `AMD` i `Intel` posiadają kontruktor z adnotacją `@Inject`

In [ ]:
public class AMD implements CPU {

    @Inject
    public AMD() {}

    @Override
    public String name() {
        return "amd";
    }
}

public class Intel implements CPU {

    @Inject
    public Intel() {}

    @Override
    public String name() {
        return "intel";
    }
}

Teraz `Dagger2` wie jak stworzyć te dwie klasy, jednak w konstruktorze klasy `Computer` przyjmujemy argument o typie interfejsu `CPU`, więc musimy jescze wskazać w odpowiednim module sposób inicjalizacji `CPU`. potrzebujemy dwa moduły - dla każdego `CPU`.

In [ ]:
@Module
public class AmdModule {
    @Provides
    CPU provideCpu(AMD cpu){
        return cpu;
    }
}

Nasz `AmdModule` zawiera jedną metodę `providesCpu` z adnotacją `@Provides`. Metoda przyjmuje jeden argument typu `AMD` i zwraca obiekt typu `CPU`.

W interfejsie `ComputerComponent` jako argument adnotacji `@Component`, podejemy listę wszystkich modułów - możemy podać tylko **jeden** moduł klasy rozszerzającej interfejs `CPU`.

In [ ]:
@Component(modules = AmdModule.class)
public interface ComputerComponent {
    void inject(MainActivity mainActivity);
}

Główna aktywność pozostaje bez zmian od ostatniego przykładu

In [ ]:
public class MainActivity extends AppCompatActivity {

    @Inject
    public Computer computer;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        ComputerComponent component = DaggerComputerComponent.create();
        component.inject(this);
        TextView textView = findViewById(R.id.textview);
        textView.setText(computer.cpuName());
    }
}

Jeżeli chcemy wykorzystać `IntelModule`

In [ ]:
@Module
public class IntelModule {
    @Provides
    CPU provideIntel(Intel cpu){
        return cpu;
    }
}

Musimy zamienić nazwę modułu w intefejsie `ComputerComponent`

In [ ]:
@Component(modules = IntelModule.class)
public interface ComputerComponent {
    void inject(MainActivity mainActivity);
}

### **`@Binds`**

Ponieważ w klasie `AmdModule` posiadamy jedną metodę, zwracającą przesłany argument, możemy zoptymalizować kod i uniknąć tworzenia jej instancji. W tym celu zmienimy klasę na klasę abstrakcyjną, oraz metodę na metodę abstrakcyjną (bez ciała).

In [ ]:
@Module
abstract public class AmdModule {
    @Binds
    abstract CPU provideCpu(AMD cpu);
}